# Entrenamiento del modelo GPT-2

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the following line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl 

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "luisangel.motta@alumnos.viu.es"
!git config --global user.name "AngelMV97"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
# Cargue del dataset
raw_datasets = load_dataset("LuangMV97/EmpathetiCounseling_Dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/47986 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12014 [00:00<?, ? examples/s]

In [ ]:
for key in raw_datasets["train"][0]:
    print(f"{key.upper()}: {raw_datasets['train'][0][key]}")

INPUT: My friend just told me she likes me as more than a friend.
LABEL: That is a tough situation. You don't want to mess up a great friendship and you also don't want to hurt her feelings either if you don't feel the same. 


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling

context_length = 512
#checkpoint = "openai-community/gpt2"
checkpoint = "LuangMV97/GPT2_EmpAI"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/924 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(text=example["input"],
                     text_target=example["label"],
                     truncation=True,
                     padding='max_length',
                     max_length=context_length,
                     return_length=True)
# Tokenización del conjunto de datos
tokenized_datasets = raw_datasets.map(tokenize_function,
                                      batched=True,
                                      remove_columns=raw_datasets["train"].column_names)

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

Map:   0%|          | 0/47986 [00:00<?, ? examples/s]

Map:   0%|          | 0/12014 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 47986
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 12014
    })
})

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="GPT2_EmpAI_FineTuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    auto_find_batch_size=True,      # Evita CUDA Out-of-Memory errors.
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=1e-2,
    learning_rate=1e-5,
    warmup_steps=8_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Pruebas de entrenamiento del modelo

In [ ]:
trainer.train() #weight_decay=1e-2, warmup_steps=8000, gpt2_EmpAI_FineTuned

Epoch,Training Loss,Validation Loss
0,2.601500,2.780482
2,2.399800,2.754276


TrainOutput(global_step=17994, training_loss=2.531824878312302, metrics={'train_runtime': 3967.5407, 'train_samples_per_second': 36.284, 'train_steps_per_second': 4.535, 'total_flos': 3.7612465422336e+16, 'train_loss': 2.531824878312302, 'epoch': 3.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 15.71


In [ ]:
eval_results

{'eval_loss': 2.7542762756347656,
 'eval_runtime': 111.5917,
 'eval_samples_per_second': 107.66,
 'eval_steps_per_second': 26.92,
 'epoch': 3.0}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712843695.9eb4b440bf21.635.0:   0%|          | 0.00/13.4k [00:00<?, ?B/s]

events.out.tfevents.1712847827.9eb4b440bf21.635.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/GPT2_EmpAI_FineTuned/commit/2c7b872682b1ffd40c602cc9614a7567af328936', commit_message='End of training', commit_description='', oid='2c7b872682b1ffd40c602cc9614a7567af328936', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=1.1, learning_rate=1e-5, gpt2_EmpAI

Epoch,Training Loss,Validation Loss
0,3.172100,3.046828
2,2.834000,2.838273


Epoch,Training Loss,Validation Loss
0,3.172100,3.046828
2,2.834000,2.838273
4,2.658400,2.791324


TrainOutput(global_step=29990, training_loss=2.9380437684957488, metrics={'train_runtime': 6549.9636, 'train_samples_per_second': 36.631, 'train_steps_per_second': 4.579, 'total_flos': 6.2687093981184e+16, 'train_loss': 2.9380437684957488, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 16.30


In [ ]:
eval_results

{'eval_loss': 2.7913241386413574,
 'eval_runtime': 108.6236,
 'eval_samples_per_second': 110.602,
 'eval_steps_per_second': 27.655,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1712681033.d6638fee8143.1883.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712674347.d6638fee8143.1883.0:   0%|          | 0.00/19.1k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/GPT2_EmpAI/commit/71c6cbaca299825e7f42293230506e1daeb7d6ad', commit_message='End of training', commit_description='', oid='71c6cbaca299825e7f42293230506e1daeb7d6ad', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0, learning_rate=1e-5, 1024, gpt2

Epoch,Training Loss,Validation Loss
1,3.408000,3.011936
2,2.951500,2.870454


Epoch,Training Loss,Validation Loss
1,3.408000,3.011936
2,2.951500,2.870454
3,2.810100,2.770332
4,2.555400,2.681702
5,2.361200,2.625288


TrainOutput(global_step=3750, training_loss=2.8017508138020832, metrics={'train_runtime': 6164.3644, 'train_samples_per_second': 38.932, 'train_steps_per_second': 0.608, 'total_flos': 6.270747475968e+16, 'train_loss': 2.8017508138020832, 'epoch': 5.0})

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 13.81


In [ ]:
eval_results

{'eval_loss': 2.625288248062134,
 'eval_runtime': 117.7865,
 'eval_samples_per_second': 101.896,
 'eval_steps_per_second': 12.743,
 'epoch': 5.0}

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1712496885.4415ec95e819.687.0:   0%|          | 0.00/7.98k [00:00<?, ?B/s]

events.out.tfevents.1712503190.4415ec95e819.687.1:   0%|          | 0.00/359 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/LuangMV97/GPT2_EmpAI/commit/ceac0a83de41c725e3c74aa78e19afef1ec70b3b', commit_message='End of training', commit_description='', oid='ceac0a83de41c725e3c74aa78e19afef1ec70b3b', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.train() #weight_decay=0, learning_rate=1e-5, 1024

Epoch,Training Loss,Validation Loss
0,0.115400,0.116726
1,0.103700,0.109285
2,0.097800,0.106526
3,0.095700,0.105459


Epoch,Training Loss,Validation Loss
0,0.115400,0.116726
1,0.103700,0.109285
2,0.097800,0.106526
3,0.095700,0.105459
4,0.094000,0.105388


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=11980, training_loss=0.1789425644532269, metrics={'train_runtime': 20917.1671, 'train_samples_per_second': 18.328, 'train_steps_per_second': 0.573, 'total_flos': 2.00331555766272e+17, 'train_loss': 0.1789425644532269, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.10538791120052338,
 'eval_runtime': 232.565,
 'eval_samples_per_second': 51.727,
 'eval_steps_per_second': 6.467,
 'epoch': 5.0}